In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('survey_results_public.csv')

In [12]:
required_data = pd.read_csv("required_data.csv")

In [13]:
df['DatabaseHaveWorkedWith'] = df['DatabaseHaveWorkedWith'].str.split(';')
df['DatabaseWantToWorkWith'] = df['DatabaseWantToWorkWith'].str.split(';')

In [14]:
df[['DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith']].head(5)

,DatabaseHaveWorkedWith,DatabaseWantToWorkWith
0,NaN,NaN
1,[Supabase],"[Firebase Realtime Database, Supabase]"
2,NaN,NaN
3,"[PostgreSQL, Redis]","[PostgreSQL, Redis]"
4,"[BigQuery, Elasticsearch, MongoDB, PostgreSQL]","[Elasticsearch, MongoDB, PostgreSQL, Redis, Su..."


In [15]:
DBdf = df[['ResponseId', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith']]

In [16]:
DBdf = required_data.merge(DBdf, how='left', on='ResponseId')

In [17]:
DBWorkedWithDF = DBdf[['ResponseId', 'DatabaseHaveWorkedWith']]

In [18]:
DBWorkedWithDF = DBWorkedWithDF.explode('DatabaseHaveWorkedWith')

In [19]:
DBWorkedWithDF['DatabaseHaveWorkedWith'] = DBWorkedWithDF['DatabaseHaveWorkedWith'].str.replace(r"\[|\]", "", regex=True)

In [20]:
DB_count = DBWorkedWithDF['DatabaseHaveWorkedWith'].value_counts()

In [21]:
DB_count.head()

DatabaseHaveWorkedWith
PostgreSQL              34892
MySQL                   31471
SQLite                  23664
MongoDB                 19546
Microsoft SQL Server    19497
Name: count, dtype: int64

In [22]:
DBWorkedWithDF['DatabaseHaveWorkedWith'].unique()

array(['Supabase', nan, 'PostgreSQL', 'Redis', 'BigQuery',
       'Elasticsearch', 'MongoDB', 'Cloud Firestore', 'MariaDB',
       'Microsoft SQL Server', 'MySQL', 'SQLite', 'Dynamodb', 'Cosmos DB',
       'Cassandra', 'H2', 'Oracle', 'Datomic', 'InfluxDB',
       'Firebase Realtime Database', 'Microsoft Access', 'Firebird',
       'IBM DB2', 'Solr', 'Cockroachdb', 'Snowflake', 'Couch DB',
       'RavenDB', 'Neo4J', 'Clickhouse', 'DuckDB', 'TiDB', 'Couchbase'],
      dtype=object)

In [23]:
result = DBWorkedWithDF.merge(required_data, how='left', on='ResponseId')

In [25]:
employedDBcount = result[result['MainBranch'] == 'Professional coder'].groupby(by='DatabaseHaveWorkedWith', as_index=False, sort=False)['ResponseId'].count()
employedDBcount.nlargest(10, 'ResponseId')

,DatabaseHaveWorkedWith,ResponseId
1,PostgreSQL,29626
9,MySQL,24491
10,SQLite,18209
8,Microsoft SQL Server,16498
5,MongoDB,15482
2,Redis,14029
7,MariaDB,10680
4,Elasticsearch,9252
11,Dynamodb,6218
15,Oracle,6069


In [26]:
studentDBcount = result[result['Employment'] == 'Student'].groupby(by='DatabaseHaveWorkedWith', as_index=False, sort=False)['ResponseId'].count()
studentDBcount.nlargest(10, 'ResponseId')

,DatabaseHaveWorkedWith,ResponseId
1,MySQL,3524
3,SQLite,2790
7,PostgreSQL,2628
0,MongoDB,2428
2,MariaDB,1344
5,Microsoft SQL Server,837
8,Redis,795
4,Cloud Firestore,777
12,Firebase Realtime Database,732
6,Oracle,493


In [27]:
DBInDevType =  result[result['DevType'] == 'Data or business analyst'].groupby(by='DatabaseHaveWorkedWith', as_index=False, sort=False)['ResponseId'].count()
DBInDevType.nlargest(10, 'ResponseId')

,DatabaseHaveWorkedWith,ResponseId
4,Microsoft SQL Server,347
5,MySQL,299
3,PostgreSQL,269
6,SQLite,199
2,Oracle,160
7,Microsoft Access,117
0,BigQuery,99
10,MariaDB,96
11,MongoDB,89
1,Snowflake,52


In [28]:
DBWantToWorkDF = df[['ResponseId', 'DatabaseWantToWorkWith']]

In [29]:
DBWantToWorkDF = DBWantToWorkDF.explode('DatabaseWantToWorkWith')

In [30]:
DBWantToWorkDF['DatabaseWantToWorkWith'] = DBWantToWorkDF['DatabaseWantToWorkWith'].str.replace("r\[\]", "", regex=True)

In [31]:
desireDB_count = DBWantToWorkDF['DatabaseWantToWorkWith'].value_counts()

In [32]:
desireDB_count.head()

DatabaseWantToWorkWith
PostgreSQL    31550
MySQL         17769
SQLite        16933
Redis         16930
MongoDB       16533
Name: count, dtype: int64

In [33]:
result1 = DBWantToWorkDF.merge(required_data, how='left', on='ResponseId')

In [34]:
employedDBdesire_count = result1[result1['Employment'] == 'Employed'].groupby(by='DatabaseWantToWorkWith', as_index=False, sort=False)['ResponseId'].count()
employedDBdesire_count.nlargest(10, 'ResponseId')

,DatabaseWantToWorkWith,ResponseId
2,PostgreSQL,23824
3,Redis,12996
17,MySQL,12177
12,SQLite,11637
5,MongoDB,11484
14,Microsoft SQL Server,9578
4,Elasticsearch,8069
13,MariaDB,5708
16,Dynamodb,4683
7,BigQuery,2893


In [35]:
studentDBdesire_count = result1[result1['Employment'] == 'Student'].groupby(by='DatabaseWantToWorkWith', as_index=False, sort=False)['ResponseId'].count()
studentDBdesire_count.nlargest(10, 'ResponseId')

,DatabaseWantToWorkWith,ResponseId
11,PostgreSQL,2578
2,MySQL,2085
1,MongoDB,2041
7,SQLite,1899
6,Redis,1270
3,MariaDB,839
16,Firebase Realtime Database,704
15,Supabase,676
0,Cloud Firestore,633
18,Cassandra,457


In [41]:
DBWorkedWithDF.to_csv('DBHaveWorkedWith.csv')

In [42]:
DBdf = df[["DatabaseHaveWorkedWith", "DatabaseWantToWorkWith", "ResponseId"]]

In [43]:
DBdf = DBdf.dropna()

In [44]:
def process_DB(row):
    list1 = row['DatabaseHaveWorkedWith']
    list2 = row['DatabaseWantToWorkWith']
    list3 = set(list1) & set(list2)
    return list(list3)

DBdf['continueDB'] = DBdf.apply(process_DB, axis=1)

In [45]:
def process_DB(row):
    list1 = row['DatabaseHaveWorkedWith']
    list2 = row['DatabaseWantToWorkWith']
    list3 = set(list2) - set(list1)
    return list(list3)

DBdf['newDB'] = DBdf.apply(process_DB, axis=1)

In [46]:
DBdf

,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,ResponseId,continueDB,newDB
1,[Supabase],"[Firebase Realtime Database, Supabase]",2,[Supabase],[Firebase Realtime Database]
3,"[PostgreSQL, Redis]","[PostgreSQL, Redis]",4,"[PostgreSQL, Redis]",[]
4,"[BigQuery, Elasticsearch, MongoDB, PostgreSQL]","[Elasticsearch, MongoDB, PostgreSQL, Redis, Su...",5,"[MongoDB, PostgreSQL, Elasticsearch]","[Supabase, Redis]"
6,"[MariaDB, Microsoft SQL Server, MySQL, Postgre...",[Datomic],7,[],[Datomic]
7,"[Dynamodb, MongoDB, MySQL, Redis, SQLite]","[BigQuery, Cockroachdb, DuckDB, Elasticsearch,...",8,"[MongoDB, Redis, SQLite]","[Snowflake, Cockroachdb, PostgreSQL, DuckDB, E..."
...,...,...,...,...,...
89178,[Dynamodb],[Dynamodb],89179,[Dynamodb],[]
89179,"[MongoDB, MySQL, PostgreSQL, SQLite]","[PostgreSQL, SQLite]",89180,"[PostgreSQL, SQLite]",[]
89180,[PostgreSQL],[PostgreSQL],89181,[PostgreSQL],[]
89182,"[MariaDB, Microsoft SQL Server, MongoDB, MySQL...","[Cassandra, Cosmos DB, Dynamodb, MariaDB, Micr...",89183,"[Microsoft SQL Server, Redis, MariaDB, SQLite,...","[PostgreSQL, Cassandra, Dynamodb, Cosmos DB, O..."


In [47]:
DBUse = DBdf[['continueDB', 'ResponseId']]
DBUse = DBUse.explode('continueDB')

In [48]:
DBUse = DBUse.groupby(by='continueDB', as_index = False)['ResponseId'].count()

In [49]:
desireDB = DBdf[['newDB', 'ResponseId']]
desireDB = desireDB.explode('newDB')

In [50]:
desireDB = desireDB.groupby(by='newDB', as_index=False)['ResponseId'].count()

In [56]:
DatabaseTable = DBUse.merge(desireDB,  how='inner', left_on='continueDB', right_on='newDB')

In [57]:
DatabaseTable = DatabaseTable[['continueDB', 'ResponseId_x', 'ResponseId_y']]
DatabaseTable.rename(columns={
    'continueDB':'databases',
    'ResponseId_x':'CurrentlyUsed',
    'ResponseId_y':'DesiredToUse'
}, inplace=True)
DatabaseTable

,databases,CurrentlyUsed,DesiredToUse
0,BigQuery,1745,1898
1,Cassandra,713,2960
2,Clickhouse,470,814
3,Cloud Firestore,2573,1093
4,Cockroachdb,399,1492
5,Cosmos DB,1370,1343
6,Couch DB,280,695
7,Couchbase,222,345
8,Datomic,172,433
9,DuckDB,308,757


In [58]:
DatabaseTable.to_csv('Databases.csv')

In [59]:
difference = DBUse.merge(desireDB, how='inner', left_on='continueDB', right_on='newDB')

In [60]:
difference.nlargest(10, 'ResponseId_y')

,continueDB,ResponseId_x,newDB,ResponseId_y
24,PostgreSQL,24896,PostgreSQL,6654
26,Redis,10935,Redis,5995
20,MongoDB,10797,MongoDB,5736
11,Elasticsearch,5129,Elasticsearch,4615
27,SQLite,13960,SQLite,2973
1,Cassandra,713,Cassandra,2960
30,Supabase,1319,Supabase,2624
10,Dynamodb,3395,Dynamodb,2178
12,Firebase Realtime Database,2258,Firebase Realtime Database,1901
0,BigQuery,1745,BigQuery,1898


In [61]:
difference.nlargest(10, 'ResponseId_x')

,continueDB,ResponseId_x,newDB,ResponseId_y
24,PostgreSQL,24896,PostgreSQL,6654
21,MySQL,15987,MySQL,1782
27,SQLite,13960,SQLite,2973
26,Redis,10935,Redis,5995
20,MongoDB,10797,MongoDB,5736
19,Microsoft SQL Server,10662,Microsoft SQL Server,949
17,MariaDB,7173,MariaDB,1144
11,Elasticsearch,5129,Elasticsearch,4615
10,Dynamodb,3395,Dynamodb,2178
23,Oracle,2705,Oracle,832
